In [1]:
#cell 1 - load obt 
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns 

# 加载本地.env文件
load_dotenv()

# 从环境变量读取连接信息
USER = os.getenv("DB_USER")
PASSWORD = os.getenv("DB_PASS")
HOST = os.getenv("DB_HOST","localhost")
PORT = os.getenv("DB_PORT","5432")
DB = os.getenv("DB_NAME")

# 检查读取是否成功(调试用)
if not PASSWORD:
    print("警告:未找到数据库密码,请检查.env文件")
else:
    print("配置加载成功(具体信息已隐藏)")

engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}")
df_rfm = pd.read_sql("SELECT * FROM analysis.analysis_user_rfm", engine)

# 基础检查
df_rfm.shape, df_rfm.describe()

配置加载成功(具体信息已隐藏)


((93350, 17),
           frequency      monetary  avg_delay_days  severe_late_rate  \
 count  93350.000000  93350.000000    93350.000000      93350.000000   
 mean       1.033423    165.196388      -10.929788          0.053807   
 std        0.209106    226.322448        9.976076          0.224330   
 min        1.000000      0.000000     -146.000000          0.000000   
 25%        1.000000     63.050000      -16.000000          0.000000   
 50%        1.000000    107.780000      -11.000000          0.000000   
 75%        1.000000    182.547500       -6.000000          0.000000   
 max       15.000000  13664.080000      188.000000          1.000000   
 
        monetary_90d  monetary_365d  monetary_long  recency_days       r_score  \
 count  93350.000000   93350.000000   93350.000000  93350.000000  93350.000000   
 mean     163.531332     165.032519     165.196388    236.487156      2.467724   
 std      224.726393     226.169392     226.322448    152.593536      1.179642   
 min    

In [2]:
#cell 2 - analysis
# ===========================
# 1.复购率初步分析
# ===========================
# 标记复购用户
df_rfm["is_repurchase"] = (df_rfm["frequency"] > 1).astype("int8")

# 计算核心指标(复购率)
repurchase_rate = df_rfm["is_repurchase"].mean()
churn_rate = 1 - repurchase_rate

print(f"总复购率: {repurchase_rate:.2%}")


总复购率: 3.00%
